In [2]:
import os

cluster_map = {"A100":"""#SBATCH -p kisski
#SBATCH -G A100:1                    
#SBATCH --mem=20G""",
"H100": """#SBATCH -p kisski-h100
#SBATCH -G H100:1                    
#SBATCH --mem=20G""",
"CPU": """#SBATCH --partition=jupyter:cpu
#SBATCH --cpus-per-task=16
#SBATCH --mem=40G"""}

format = """#!/bin/bash
#SBATCH --job-name={dataset}
{cluster}
#SBATCH --time=12:00:00
#SBATCH --output=convert_{dataset}_%j.out
#SBATCH --error=convert_{dataset}_%j.err
#SBATCH --mail-type=FAIL # Email on start, end, failure
#SBATCH --mail-user=hao.huang@tib.eu # <-- Replace with your real email
echo "===== JOB STARTED ====="
echo "Hostname: $(hostname)"
echo "Date: $(date)"
echo "User: $USER"
# Load environment
module load miniforge3
# module load cuda/11.8
{cuda_use}module load gcc/13.2.0
{cuda_use}module load cuda/11.8

# Set up conda
source "$(conda info --base)/etc/profile.d/conda.sh"
conda activate /mnt/vast-kisski/projects/kisski-tib-activecl/cenv
# Diagnostics
echo "Which python: $(which python)"
python -c "import torch; print('Torch:', torch.__version__, '| CUDA:', torch.cuda.is_available())"
# Navigate to project directory
cd /mnt/vast-kisski/projects/kisski-tib-activecl/GRAPE-imputer
# Run scripts
echo "--- Running: run_data_prepare.py"

python -u run_grape.py --dataset {dataset} --missing_p {miss_p} 

echo "===== JOB COMPLETED ====="
echo "Date: $(date)"
"""

# List of datasets
# datasets = ['Syn', 'BlogCatalog1', 'Flickr1', 'Youtube'] # 'Youtube',
# datasets = ['Syn_M=None_SimRel=1_Rel=4',  'BlogCatalog1_M=20_SimRel=1_Rel=4', 'Flickr1_M=20_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4']
# datasets += ['Syn_M=None_SimRel=0_Rel=1',  'BlogCatalog1_M=20_SimRel=0_Rel=1', 'Flickr1_M=20_SimRel=0_Rel=1', 'Youtube_M=20_SimRel=0_Rel=1']
datasets_org = ['Syn_M=None_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4', 'BlogCatalog1_M=20_SimRel=0_Rel=1', 'Flickr1_M=20_SimRel=0_Rel=1']      # network relationships not based on similarity
datasets = [e+"_MCAR" for e in datasets_org]

miss_ps = [0.1, 0.3]

################################################################################################################################################
cluster = 'A100'        # H100, CPU
slurm_dir = f'{cluster}_slurm'
cuda_use = "# " if cluster == 'CPU' else ""
################################################################################################################################################

os.makedirs(slurm_dir, exist_ok=True)

# Create slurm files and collect sbatch commands
sbatch_commands = []

for dataset in datasets:
    for miss_p in miss_ps:
        # Create filename
        filename = os.path.join(slurm_dir, f"convert_{dataset}_{miss_p}.slurm")
        
        # Fill in the template with the dataset name
        job_content = format.format(cluster=cluster_map[cluster], dataset=dataset, cuda_use=cuda_use, 
                                    miss_p=miss_p)
        
        # Write the slurm file
        with open(filename, 'w') as f:
            f.write(job_content)
        
        # Make it executable
        os.chmod(filename, 0o755)
        
        # Add sbatch command to list
        sbatch_commands.append(f"sbatch {filename}")

print("cd /mnt/vast-kisski/projects/kisski-tib-activecl/GRAPE-imputer")
for cmd in sbatch_commands:
    print(cmd)

# Create a batch script to submit all jobs at once
batch_file = os.path.join(slurm_dir, "submit_all_jobs.sh")
with open(batch_file, 'w') as f:
    f.write("#!/bin/bash\n\n# Submit all conversion jobs\n\n")
    for cmd in sbatch_commands:
        f.write(f"{cmd}\n")

# Make batch file executable
os.chmod(batch_file, 0o755)

# print(f"\nAlternatively, run all jobs with: {batch_file}")

cd /mnt/vast-kisski/projects/kisski-tib-activecl/GRAPE-imputer
sbatch A100_slurm/convert_Syn_M=None_SimRel=1_Rel=4_MCAR_0.1.slurm
sbatch A100_slurm/convert_Syn_M=None_SimRel=1_Rel=4_MCAR_0.3.slurm
sbatch A100_slurm/convert_Youtube_M=20_SimRel=1_Rel=4_MCAR_0.1.slurm
sbatch A100_slurm/convert_Youtube_M=20_SimRel=1_Rel=4_MCAR_0.3.slurm
sbatch A100_slurm/convert_BlogCatalog1_M=20_SimRel=0_Rel=1_MCAR_0.1.slurm
sbatch A100_slurm/convert_BlogCatalog1_M=20_SimRel=0_Rel=1_MCAR_0.3.slurm
sbatch A100_slurm/convert_Flickr1_M=20_SimRel=0_Rel=1_MCAR_0.1.slurm
sbatch A100_slurm/convert_Flickr1_M=20_SimRel=0_Rel=1_MCAR_0.3.slurm
